# Kerasによる音楽生成（2018年度 DL4US #2 最終課題）

板垣 正敏（masa-ita)

# 概要

ディープラーニングによる音楽生成をテーマに、RNN（リカレントニューラルネットワーク）による予測モデル、VAE（変分オートエンコーダ）、さらにGAN（敵対的生成ネットワーク）による手法を使って実験を行った。学習した楽曲に近いメロディを生成する手法としては、RNN（LSTM）による予測モデルが単純ではあるが効果的であった。VAEによる隠れベクトル空間の構築は、楽曲の特徴空間を構成するという意味では効果的であり、楽曲同士の類似性の計測指標となるが、学習サンプルが少ないせいか、人間が聞いて音楽らしく聞こえる範囲は狭く、楽曲生成手法としては使い方が難しい。GANによる生成も試みたが、安定した学習ができなかった。

# 目的

仕事の領域では、画像やテキストの分類、数値データをもとにした回帰といった領域の問題を扱うことが多いため、今回の最終課題ではあまり取り組んだことのなかった生成系の問題を選択した。コンピュータによる音楽生成は古くから研究されている領域であるが、趣味で音楽を聴くことが多いこともあり、MIDIデータをサンプルとした音楽生成をテーマに選んだ。RNNによるメロディー生成だけではなく、VAEによる音楽モデルの構築や、GANによる音楽生成も試みて、その比較を行うことを目的とした。

# 使用するデータ

深層学習データに転用しやすいMIDIデータをサンプルとした。著作権の問題を避けるため、今回はサイト[midiworld.com](http://midiworld.com/bach.htm)にあるMIDIデータのうち、J.S.Bachの楽曲を使用した。今回は手始めということもあり、複数チャネルではなく、単一チャネルの2声のインベンション15曲を使用している。




# 手法

今回は単一の手法ではなく、下記の3つの手法で実験を行い比較検討を行った。

1. リカレントニューラルネットワーク（RNN）による学習と予測
2. 変分オートエンコーダー（VAE： Valiational Autoencoder）による生成
3. 敵対的生成ネットワーク（GAN： Generative Adversal Network)による生成

## リカレントニューラルネットワーク（RNN）による学習と予測

### 設計

 LSTMによるメロディー生成では、サイト[How to Generate Music using a LSTM Neural Network in Keras](https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5)<sup>1</sup>
を参考にし、固定長（音素数100）のメロディを入力データ $ｘ$ に、そのメロディの直後の音素をターゲット $ｙ$ とした学習を行い、生成時には与えられた初期メロディデータをシードとしたメロディ生成を行った。

![LSTMによる予測モデル](images/lstm_model_LR.png)


参考にしたサイトでは、音素を整数化したデータを正規化したものを入力データとしているが、One-Hotエンコーディングしたものを入力としている。   

### 実験

損失関数はRMSpropの既定値を用い、バッチサイズ64で200エポック学習をさせた。
損失と正解率の推移はグラフの通りである。

![LSTMモデルの損失値と正解率の推移](images/lstm_midi_loss_acc.png)

Dropoutを指定しているが、明らかに過学習が発生している。
ただし、今回の目的は、訓練用データに似た出力を生成することであるため、これ以上過学習を抑える手法は取らないこととした。

参考としたソースの手法では、曲を生成する際に、初期データとして訓練用データの中からランダムに選択したフレーズを使用している。この方法で生成したサンプルを下記のサイトにアップロードしてある。
比較対象として、訓練用データのフレーズを使用せず、ランダムに選択した音素を1つ用い、パディングにより長さを揃えたものを初期データとして曲を生成した。

### 結果

今回は、リズム要素を排除した処理を行っているが、出力された曲を聞いた限りでは、訓練用の楽曲の傾向に近いメロディが生成されているように思われる。初期データとして訓練用のデータのフレーズを使わずに生成を行った場合でも、大きな差異は発生していないと思われる。

訓練データ中のフレーズを初期値とした場合  
[https://soundcloud.com/itagakim/lstm-generator-learned-frag-1](https://soundcloud.com/itagakim/lstm-generator-learned-frag-1)  
ランダムに選択した音素をパディングし初期値とした場合  
[https://soundcloud.com/itagakim/lstm-generator-learned-single-note-1](https://soundcloud.com/itagakim/lstm-generator-learned-single-note-1)

### 考察

今回の手法では、初期データあるいはその後に生成されたメロディに基づいて、最も尤度の高い音素を選択することになる（貪欲的サンプリング）。このため、同じ初期データを与えた場合には、同じ出力が得られる。
目的は、「訓練データによく似た楽曲を生成する」ことにあるので、これでも十分であると考えられるが、よりバリエーションを与える方法として、確率的サンプリングを行う手法が考えられる。[「PythonとKerasによるディープラーニング」Francois Chollet](https://book.mynavi.jp/ec/products/detail/id=90124)<sup>2</sup>ではソフトマックス温度として手法が紹介されている。

[Olof Mogren, C-RNN-GAN: Continuous recurrent neura networks with adversarial training](https://arxiv.org/abs/1611.09904)<sup>3</sup>では、客観的評価指標としてPolyphony, Scale consistency, Repetitions, Tone spanの4つを挙げているが、今回は時間の関係で調査していない。訓練データとの類似性について評価を行う方法については、次の手法であるVariational Autoencoderのところで検討することにする。

## 変分オートエンコーダー（VAE： Valiational Autoencoder）による生成

### 設計

LSTMを使用したVAEを構成し、学習したデータから隠れ確率空間を構成することを目指した。
VAEによる音楽生成としては、Googleのmagentaプロジェクト [Magenta](https://magenta.tensorflow.org/music-vae)<sup>4</sup>があるが、今回参考にしたのは、音楽データではなくテキストデータのVAEを構成した[Text Variational Autoencoder in Keras](http://alexadam.ca/ml/2017/05/05/keras-vae.html)<sup>5</sup>の例である。

ただし、コードはDL4USのサンプルに似た形式に書き換えた。

![VAEモデル](images/vae_all_small.png)

### 実験

実験は、隠れベクトルを2次元として実施した。LSTMの場合と異なり、訓練用データには楽曲そのもの（最長音素数300）を使用し、オプティマイザにはRMSpropの既定値を使用し、1000エポックあるいは2000エポック訓練を行った。
実験を始めた当初は、訓練中の挙動が安定せず、一旦減少した損失関数値が突如上昇するなどの減少に悩まされた。

![VAEの訓練時に損失値が不安定になる例](images/vae_loss_unstable.png)

その後コードを書き換える際に、参考にしたサイトの正則化項が下記のようになっており、本来合計を取るべきところが平均になっていることに気がついた。
```python
kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
```
正則化項の式及び係数については、様々なバリエーションが存在するが、これが原因の可能性がある。
正則化項を合計に修正後は安定した訓練ができるようになった。

![VAEの訓練時に安定した損失値が得られた例](images/vae_loss_stable.png)


比較のため、LSTM層を2層にした場合と、隠れベクトルを4次元にしたものも実験した。

訓練に使用した楽曲を入力として、出力された楽曲の類似度を検討した。また、隠れベクトルにおけるもとの楽曲の分布を調べ、その分布図で確率が低いポイントをデコーダに与えた場合の出力や、比較的既存楽曲に近いポイントを与えた場合の出力についても比較を行った。

### 結果

隠れベクトル2次元の学習結果をプロットしたものを示す。

![VAEの２次元隠れ空間と各入力サンプルのｚ_mean](images/latent_2d.png)

楽曲によって平均値が異なることがわかる。また、分散が小さいせいか、確率が低いポイントだけではなく、比較的既存楽曲に近いポイントをデコーダの入力として与えた場合の出力も、人の耳ではランダム性が高く、訓練用楽曲とは異なる出力となっている。

訓練用楽曲をVAEの入力としたもの。  
BVW772: [https://soundcloud.com/itagakim/vae-with-learned-tune-bvw772](https://soundcloud.com/itagakim/vae-with-learned-tune-bvw772)  
BVW773: [https://soundcloud.com/itagakim/vae-with-learned-tune-bvw773](https://soundcloud.com/itagakim/vae-with-learned-tune-bvw773)  

隠れ空間の原点（0.0, 0.0）をデコーダの入力としたもの。  
[https://soundcloud.com/itagakim/vae-with-latent-point-0-0](https://soundcloud.com/itagakim/vae-with-latent-point-0-0)  
隠れ空間の点（0.03, -0.02）をデコーダの入力としたもの。  
[https://soundcloud.com/itagakim/vae-with-latent-003-002](https://soundcloud.com/itagakim/vae-with-latent-003-002)  

隠れベクトルが2次元の場合と、4次元の場合、また、LSTMが1層の場合と2層の場合に顕著な差は見られなかった。

### 考察


今回、隠れベクトル空間で人間の耳にもとの楽曲に近いと感じる領域が小さい結果となった理由については、学習データが極端に少ないためとも考えられる。
他方で、この学習済みのVAEのエンコーダを使って、VAE以外の手法で生成した楽曲がどれくらい訓練用の楽曲に近いかという測定ができるのではないかと考えた。
試みに、LSTMによる学習・生成で出力した楽曲をプロットしたものを下図に示す。

![LSTMで生成された楽曲の隠れベクトル](images/latent_compared.png)

## 敵対的生成ネットワーク（GAN： Generative Adversarial Network)による生成

### 設計

LSTMを使った生成ネットワークと判別ネットワークを組み合わせたGANを構成した。
最初はどちらもLSTM1層からなるネットワークである。エンコーダ、デコーダともVAEとほぼ同じ構造である。

![Generator](images/gan_generator.png)
![Discriminator](images/gan_discriminator.png)

### 実験

何回も設定を変え10,000回から20,000回のイテレーションで実験を繰り返したが、同じ音符の繰り返しといった出力で、Generatorがうまく学習できないことが多かった。DiscriminatorおよびGAN全体のオプティマイザをAdamとRMSpropで試したり、それぞれの学習率を変更したりしたが、うまく学習が進まなかった。


VAEと似たような構造であることに着目し、VAEで学習したデコーダの重みをGeneratorに移植して学習することも試した。

### 結果

[C-RNN-GAN: Continuous recurrent neural networks with adversarial training](https://arxiv.org/abs/1611.09904)<sup>6</sup>および[Improved Techniques for Training GANs](https://arxiv.org/abs/1606.03498)<sup>7</sup>の論文を参考に、GANの損失とDiscriminatorの損失を比較し、GANの損失が改善しない場合にはDiscriminatorの学習を停止したり、それぞれ相手の損失値の70%を下回ったら学習を停止するなどの手法を試みたが、効果は得られなかった。

比較的楽曲らしいサンプル  
[https://soundcloud.com/itagakim/gan-generated-5400](https://soundcloud.com/itagakim/gan-generated-5400)  
[https://soundcloud.com/itagakim/gan-generated-6400](https://soundcloud.com/itagakim/gan-generated-6400)  

VAEの重みを移植した場合でも、初期はメロディが生成されていたものの、徐々に重みが破壊され、ついには同じ音符の繰り返しとなってしまう結果となった。

### 考察

今回のモデルでは、ランダムに初期化される隠れベクトルを系列データに変換する手法として、RepeatVectorを使用し、同じものをコピーしているが、その結果として同じ音符の連続という初期値が限定される結果となり、Generatorの学習が進まないのではないかと考える。
これを回避する方法として、ランダムな隠れベクトルから、ランダムな音素系列を生成し、それを初期データとして与える方法が考えられる。しかし、Kerasに用意されている機能で、これを実現し、なおかつ1つのモデルとして誤差逆伝搬法が使えるモデルを構築することができなかった。
さらに、[C-RNN-GAN: Continuous recurrent neural networks with adversarial training](https://arxiv.org/abs/1611.09904)<sup>6</sup>の論文では、出力が連続性のある音素の構成要素（音の長さ、周波数、強さ、前の音との時間間隔）であるのに対して実験モデルの出力は離散的な音素であるため、誤差逆伝搬による変化が小さい場合、音素の変更に至らないという現象が発生しているのではないかと推測する。

# まとめ

複数の手法を試したが、時間の関係で十分な深掘りができなかった。また、いずれの手法においても訓練用データが少ないので、過学習によって汎化ができていないことが、結果に影響している可能性は否定できない。

VAEによる手法は、楽曲生成手法としても興味深いものだが、音楽の特性・構成要素を比較的低次元に特徴抽出でき、分析手法や他の手法の評価手法としても使える可能性を感じた。

GANは、離散的な出力を得る手法としては難しい。これを解決する手法として、
[Improved Techniques for Training GANs](https://arxiv.org/abs/1606.03498)<sup>7</sup>で紹介されているFeature Matchingや、強化学習で用いられるPolicy Gradientも今後試してみたい。

なお、今回作成したプログラムの一部は下記のリポジトリにアップロードしてある。

[https://github.com/masa-ita/keras-music-generators](https://github.com/masa-ita/keras-music-generators)

# 参考文献

1. [How to Generate Music using a LSTM Neural Network in Keras](https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5)  
https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
2. [「PythonとKerasによるディープラーニング」Francois Chollet](https://book.mynavi.jp/ec/products/detail/id=90124)  
https://book.mynavi.jp/ec/products/detail/id=90124
3. [Olof Mogren, C-RNN-GAN: Continuous recurrent neura networks with adversarial training](https://arxiv.org/abs/1611.09904)  
https://arxiv.org/abs/1611.09904
4. [Magenta](https://magenta.tensorflow.org/music-vae)  
https://magenta.tensorflow.org/music-vae
5. [Text Variational Autoencoder in Keras](http://alexadam.ca/ml/2017/05/05/keras-vae.html)  
http://alexadam.ca/ml/2017/05/05/keras-vae.html
6. [C-RNN-GAN: Continuous recurrent neural networks with adversarial training](https://arxiv.org/abs/1611.09904)  
https://arxiv.org/abs/1611.09904
7. [Improved Techniques for Training GANs<sup>7</sup>](https://arxiv.org/abs/1606.03498)  
https://arxiv.org/abs/1606.03498